In [2]:
import numpy as np
import numpy as np
import pandas as pd
from utils.data_preprocess_utils import get_config
import utils.dataset_generate_utils as gen
import utils.data_preprocess_utils as pre
import warnings
import pandas as pd
from easydict import EasyDict
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
# ignore warnings
warnings.filterwarnings("ignore")

In [3]:
def process_day(day):
    try:
        day_str = str(day).split(' ')[0]
        df = pd.read_feather(files_dict[day_str])
        df.sort_values(by=['MMSI', 'time'], inplace=True)
        df['inout'] = 0
        df['label'] = -1
        df = df.groupby('MMSI').apply(gen.get_traj_point_type, cfg)
        day_dic = gen.generate_dataset(day, cfg, df)
        file_name = cfg.out_dir + day_str + '.npy'
        np.save(file_name, day_dic)
    except Exception as e:
        print(f"Error processing day {day}: {e}")
    

Q1 = pd.date_range('2019-01-01', periods=1, freq='1D')
cfg = EasyDict(get_config('./cfg/dataset_generate_cfg.yaml'))
files = pre.get_all_files(cfg.in_dir)
files_dict = pre.files2dict(files)
with ProcessPoolExecutor(max_workers=8) as executor:
    # Create a list of futures for each day
    futures = [executor.submit(process_day, day) for day in Q1]

    # Display progress using tqdm
    for _ in tqdm(as_completed(futures), total=len(futures)):
        pass  # Do nothing here as we are using tqdm for progress display

# Ensure all tasks are completed
for future in futures:
    future.result()

100%|██████████| 1/1 [01:09<00:00, 69.04s/it]
